In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pickle
import json

In [3]:
with open('answer_data_annotations.json', 'r') as f:
    answer_annotations = json.load(f)

with open('wrong_data_annotations.json', 'r') as f:
    wrong_annotations = json.load(f)

In [4]:
answer_data = 'answer_annotated_data.csv'
wrong_data = 'wrong_annotated_data.csv'

In [5]:
answer_data = pd.read_csv(answer_data)
wrong_data = pd.read_csv(wrong_data)

In [6]:
answer_data['Label'] = 1  # 정답 데이터 ==> label 1
wrong_data['Label'] = 0  # 오답 데이터 ==> label 0

In [7]:
def get_video_data(data, annotations):
    video_data = []
    for video in annotations['videos']:
        frames = [int(frame['frame'].split('_')[-1].split('.')[0]) for frame in video['annotations']]
        video_data.append(data.loc[frames].reset_index(drop=True))  # 인덱스 리셋
    return video_data

In [9]:
answer_videos = get_video_data(answer_data, answer_annotations)
wrong_videos = get_video_data(wrong_data, wrong_annotations)

In [10]:
all_videos = answer_videos + wrong_videos

In [11]:
scaler = MinMaxScaler()

In [14]:
def scale_videos(videos, scaler):
    scaled_videos = []
    for video in videos:
        features = video.select_dtypes(include=[np.number]).columns.difference(['Label'])  # 숫자형 데이터만 select
        scaled_data = scaler.fit_transform(video[features])
        scaled_video = pd.DataFrame(scaled_data, columns=features)
        scaled_video['Label'] = video['Label'].values
        scaled_videos.append(scaled_video)
    return scaled_videos

In [15]:
scaled_videos = scale_videos(all_videos, scaler)

In [17]:
def create_dataset_rf(videos, time_step=10):
    dataX, dataY = [], [] # dataX가 flatten된 시계열 데이터, dataY가 레이블
    for video in videos:
        dataset = video.drop(columns=['Label']).values
        labels = video['Label'].values
        for i in range(len(dataset) - time_step):
            a = dataset[i:(i + time_step)].flatten() ## i에서 time_step 까지의 데이터 가져오고 flatten
            dataX.append(a)
            dataY.append(labels[i + time_step]) # 레이블은 바로 다음 레이블 입력. 시계열 데이터 특성 반영
    return np.array(dataX), np.array(dataY)

In [18]:
time_step = 10 # 10개 연속된 데이터 설정.
X_rf, y_rf = create_dataset_rf(scaled_videos, time_step)

In [19]:
## 기존 trian, test, vali 셋 구분지어 놓은 것, train, vali 셋으로만 분리 수정

# X_train_rf, X_temp_rf, y_train_rf, y_temp_rf = train_test_split(X_rf, y_rf, test_size=0.4, random_state=42, stratify=y_rf) # stratify로 각 클래스 비율 유지.
# X_val_rf, X_test_rf, y_val_rf, y_test_rf = train_test_split(X_temp_rf, y_temp_rf, test_size=0.5, random_state=42, stratify=y_temp_rf)

X_train_rf, X_val_rf, y_train_rf, y_val_rf = train_test_split(X_rf, y_rf, test_size=0.4, random_state=42, stratify=y_rf)

In [20]:
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train_rf, y_train_rf)

RandomForestClassifier(random_state=42)

In [21]:
# vali set
y_val_pred_rf = model_rf.predict(X_val_rf)
accuracy_val_rf = accuracy_score(y_val_rf, y_val_pred_rf)
report_val_rf = classification_report(y_val_rf, y_val_pred_rf)

In [ ]:
print(f'Vali set Accuracy: {accuracy_val_rf}')
print(f'Vali set Report:\n{report_val_rf}')

In [22]:
# # test set 별도 처리 수정
# y_test_pred_rf = model_rf.predict(X_test_rf)
# accuracy_test_rf = accuracy_score(y_test_rf, y_test_pred_rf)
# report_test_rf = classification_report(y_test_rf, y_test_pred_rf)


test_data = pd.read_csv('test_annotated_data.csv')  
test_annotations = json.load(open('test_data_annotations.json', 'r'))

In [ ]:
###

test_videos = get_video_data(test_data, test_annotations)
scaled_test_videos = scale_videos(test_videos, scaler)
X_test_rf, y_test_rf = create_dataset_rf(scaled_test_videos, time_step)

In [ ]:
### test 셋 성능

y_test_pred_rf = model_rf.predict(X_test_rf)
accuracy_test_rf = accuracy_score(y_test_rf, y_test_pred_rf)
report_test_rf = classification_report(y_test_rf, y_test_pred_rf)

In [ ]:
print(f'Test Accuracy: {accuracy_test_rf}')
print(f'Test Report:\n{report_test_rf}')

In [23]:
# print(f'Validation Accuracy: {accuracy_val_rf}')
# print(f'Validation Classification Report:\n{report_val_rf}')

# print('\n')

# print(f'Test Accuracy: {accuracy_test_rf}')
# print(f'Test Classification Report:\n{report_test_rf}')

Validation Accuracy: 1.0
Validation Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        28

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



Test Accuracy: 1.0
Test Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        28

    accuracy                           1.00        39
   macro avg       1.00      1.00      1.00        39
weighted avg       1.00      1.00      1.00        39



In [24]:
with open('running_posture_model_edit_frame.pkl', 'wb') as model_file:
    pickle.dump(model_rf, model_file)

In [25]:
with open('data_scaler_edit_frame.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)